# Recompra Bázica

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as px


# Evitar exibir Warnings
# import warnings
# warnings.filterwarnings("ignore")


## Base de dados

In [93]:
base_compras = pd.read_csv('data-set/vendas_de_produtos.csv', dtype={'ID_Cliente': 'Int64', 'ID_Produto': 'Int64'})

### Exploração dos dados 

In [94]:
base_compras

,ID_Cliente,Data,ID_Produto,Descrição_Produto,Quantidade,Preço_Unitário,ID_Pedido,Desconto,Frete,Total_do_Pedido
0,12429082030,2021-05-27,14,Bázica Gola C - Air - Azul Marinho - M,1,109.0,1000000401,0.0,4.90,113.90
1,12793619292,2021-05-27,<NA>,Bázica Gola C - Powerful - Preto - P,1,109.0,1000000400,0.0,4.90,113.90
2,1,2021-05-27,<NA>,Bázica Gola C - Powerful - Preto - M,1,109.0,1000000399,0.0,4.90,113.90
3,2,2021-05-26,<NA>,Bázica Gola C - Powerful - Preto - M,2,109.0,1000000398,0.0,4.90,301.90
4,2,2021-05-26,<NA>,Bázica Gola C - Powerful - Branco - M,1,109.0,1000000398,0.0,4.90,301.90
...,...,...,...,...,...,...,...,...,...,...
17023,15990665892,2022-12-31,2,Bázica Gola C - Preta - Powerful - M,1,119.0,9017,0.0,21.19,140.19
17024,15896646324,2022-12-31,192,Bázica Vibe - Azul Cobalto - Viscose - M,1,223.0,9018,0.0,0.00,223.00
17025,12268398905,2022-12-31,211,Bázica Lord - Branca - Pima - GG,1,247.0,9019,0.0,0.00,247.00
17026,15990672946,2022-12-31,461,Bázica Long - Ocean - Air - G,1,119.0,9020,23.8,0.00,214.20


**Verificando Informações da Base de Dados**

In [95]:
base_compras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17028 entries, 0 to 17027
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID_Cliente         17026 non-null  Int64  
 1   Data               17028 non-null  object 
 2   ID_Produto         16625 non-null  Int64  
 3   Descrição_Produto  17028 non-null  object 
 4   Quantidade         17028 non-null  int64  
 5   Preço_Unitário     16804 non-null  float64
 6   ID_Pedido          17028 non-null  int64  
 7   Desconto           16984 non-null  float64
 8   Frete              17028 non-null  float64
 9   Total_do_Pedido    17028 non-null  float64
dtypes: Int64(2), float64(4), int64(2), object(2)
memory usage: 1.3+ MB


**Verificando a soma de dados ausentes em cada coluna**

In [96]:
base_compras.isnull().sum()

ID_Cliente             2
Data                   0
ID_Produto           403
Descrição_Produto      0
Quantidade             0
Preço_Unitário       224
ID_Pedido              0
Desconto              44
Frete                  0
Total_do_Pedido        0
dtype: int64

In [97]:
np.round(base_compras.isnull().sum()['ID_Cliente']/ base_compras.shape[0] * 100, 2)

0.01

Como é percebido, existem dois pedidos com id de cliente nulos que equivalem a $0.01$%, então irei analisar se existem outros pedidos de venda com o mesmo código do pedido para subtituir o ID de cliente nulo desses pedidos,  caso contrário irei optar por remover esses dados devido que a quantidade pedidos não irá influenciar tanto no resultado final.

**Verificando os ID de Clientes Nulos**

In [98]:
base_compras[base_compras['ID_Cliente'].isnull()]

,ID_Cliente,Data,ID_Produto,Descrição_Produto,Quantidade,Preço_Unitário,ID_Pedido,Desconto,Frete,Total_do_Pedido
483,<NA>,2020-09-10,<NA>,Bázica Gola C - Powerful - Branco - G,1,NaN,1000000044,0.0,0.0,109.0
485,<NA>,2020-08-30,<NA>,Bázica Gola C - Powerful - Preto - GG,1,NaN,1000000039,0.0,0.0,197.0


**Verificando se existem outra venda com número de pedido igual**

In [99]:
base_compras[base_compras['ID_Pedido'] == 1000000044]

,ID_Cliente,Data,ID_Produto,Descrição_Produto,Quantidade,Preço_Unitário,ID_Pedido,Desconto,Frete,Total_do_Pedido
483,<NA>,2020-09-10,<NA>,Bázica Gola C - Powerful - Branco - G,1,NaN,1000000044,0.0,0.0,109.0


In [100]:
base_compras[base_compras['ID_Pedido'] == 1000000039]

,ID_Cliente,Data,ID_Produto,Descrição_Produto,Quantidade,Preço_Unitário,ID_Pedido,Desconto,Frete,Total_do_Pedido
485,<NA>,2020-08-30,<NA>,Bázica Gola C - Powerful - Preto - GG,1,NaN,1000000039,0.0,0.0,197.0


**Como não existe outra venda com mesmo código de pedido essas linhas serão removidas**  

In [101]:
base_compras = base_compras.dropna(subset=['ID_Cliente'])
base_compras.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17026 entries, 0 to 17027
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID_Cliente         17026 non-null  Int64  
 1   Data               17026 non-null  object 
 2   ID_Produto         16625 non-null  Int64  
 3   Descrição_Produto  17026 non-null  object 
 4   Quantidade         17026 non-null  int64  
 5   Preço_Unitário     16804 non-null  float64
 6   ID_Pedido          17026 non-null  int64  
 7   Desconto           16982 non-null  float64
 8   Frete              17026 non-null  float64
 9   Total_do_Pedido    17026 non-null  float64
dtypes: Int64(2), float64(4), int64(2), object(2)
memory usage: 1.5+ MB


**Verificando os dtypes das colunas**

In [117]:
base_compras.dtypes

ID_Cliente                    Int64
Data                 datetime64[ns]
ID_Produto                    Int64
Descrição_Produto            object
Quantidade                    int64
Preço_Unitário              float64
ID_Pedido                     int64
Desconto                    float64
Frete                       float64
Total_do_Pedido             float64
dtype: object

**Informações sobre as datas**

In [110]:
base_compras.head()

,ID_Cliente,Data,ID_Produto,Descrição_Produto,Quantidade,Preço_Unitário,ID_Pedido,Desconto,Frete,Total_do_Pedido
0,12429082030,2021-05-27,14,Bázica Gola C - Air - Azul Marinho - M,1,109.0,1000000401,0.0,4.9,113.9
1,12793619292,2021-05-27,<NA>,Bázica Gola C - Powerful - Preto - P,1,109.0,1000000400,0.0,4.9,113.9
2,1,2021-05-27,<NA>,Bázica Gola C - Powerful - Preto - M,1,109.0,1000000399,0.0,4.9,113.9
3,2,2021-05-26,<NA>,Bázica Gola C - Powerful - Preto - M,2,109.0,1000000398,0.0,4.9,301.9
4,2,2021-05-26,<NA>,Bázica Gola C - Powerful - Branco - M,1,109.0,1000000398,0.0,4.9,301.9


In [120]:
pd.DataFrame(base_compras.Data.describe())

C:\Users\Kaiki\AppData\Local\Temp\ipykernel_1288\1134688599.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  pd.DataFrame(base_compras.Data.describe())


,Data
count,17026
unique,636
top,2022-10-14 00:00:00
freq,484
first,2020-08-19 00:00:00
last,2022-12-31 00:00:00


Vemos que as compras online foram feitas no perido de 14-10-2022 a 31-12-2022

### Prevendo compra do cliente